In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import roc_auc_score
from pathlib import Path
from tqdm.auto import tqdm
from scipy.stats import skew, kurtosis
import warnings

warnings.filterwarnings("ignore")

# ====================
# 1. Load Dataset
# ====================
data_folder = Path("./39_Training_Dataset")
train_info_path = data_folder / "train_info.csv"
train_data_folder = data_folder / "train_data"

info = pd.read_csv(train_info_path)
print("Train Info Shape:", info.shape)

Train Info Shape: (1955, 8)


In [2]:
# Functions to load and process data
def load_sensor_data(filepath):
    data = []
    with open(filepath, "r") as f:
        for idx, line in enumerate(f.readlines()):
            if idx == 0 or line.strip() == "":
                continue
            parts = line.strip().split()
            if len(parts) >= 6:
                data.append([int(x) for x in parts[:6]])
    return np.array(data)

def segment_swings(sensor_data, n_swings=27):
    swing_indices = np.linspace(0, len(sensor_data), n_swings+1, dtype=int)
    swings = []
    for i in range(n_swings):
        swings.append(sensor_data[swing_indices[i]:swing_indices[i+1]])
    return swings

def extract_features_from_swing(swing_data):
    features = {}
    for i, axis in enumerate(['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz']):
        data_axis = swing_data[:, i]
        features[f'{axis}_mean'] = np.mean(data_axis)
        features[f'{axis}_std'] = np.std(data_axis)
        features[f'{axis}_max'] = np.max(data_axis)
        features[f'{axis}_min'] = np.min(data_axis)
        features[f'{axis}_skew'] = skew(data_axis)
        features[f'{axis}_kurtosis'] = kurtosis(data_axis)
    acc_magnitude = np.linalg.norm(swing_data[:, 0:3], axis=1)
    gyro_magnitude = np.linalg.norm(swing_data[:, 3:6], axis=1)
    features['acc_mag_mean'] = np.mean(acc_magnitude)
    features['gyro_mag_mean'] = np.mean(gyro_magnitude)
    return features

def aggregate_swing_features(swing_features_df):
    session_features = {}
    for col in swing_features_df.columns:
        session_features[f'{col}_mean'] = swing_features_df[col].mean()
        session_features[f'{col}_std'] = swing_features_df[col].std()
        session_features[f'{col}_min'] = swing_features_df[col].min()
        session_features[f'{col}_max'] = swing_features_df[col].max()
    return session_features

def process_one_file(file_path):
    try:
        data = load_sensor_data(file_path)
        swings = segment_swings(data)
        swing_feats = [extract_features_from_swing(s) for s in swings]
        swing_feats_df = pd.DataFrame(swing_feats)
        session_feats = aggregate_swing_features(swing_feats_df)
        return session_feats
    except Exception as e:
        print(f"Failed processing {file_path.name}: {e}")
        return None

In [3]:
X_list = []
y_list = []

for file_path in tqdm(sorted(train_data_folder.glob("*.txt"))):
    unique_id = int(file_path.stem)
    row_info = info[info['unique_id'] == unique_id]
    if row_info.empty:
        continue
    session_feats = process_one_file(file_path)
    if session_feats is None:
        continue
    mode = int(row_info['mode'].values[0])
    for m in range(1, 11):
        session_feats[f'mode_{m}'] = 1 if mode == m else 0
    X_list.append(session_feats)
    y_list.append(row_info[['gender', 'hold racket handed', 'play years', 'level']].iloc[0])

X = pd.DataFrame(X_list)
y = pd.DataFrame(y_list)
print("✅ Finished loading dataset.", X.shape, y.shape)

  0%|          | 0/1955 [00:00<?, ?it/s]

✅ Finished loading dataset. (1955, 162) (1955, 4)


In [4]:
# ====================
# 2. Preprocessing
# ====================
y_encoded = {}
for col in ['gender', 'hold racket handed', 'play years', 'level']:
    le = LabelEncoder()
    y_encoded[col] = le.fit_transform(y[col])

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# ====================
# 3. Train Gender (Full Features)
# ====================
feature_importances = {}
folds = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=42).split(X_scaled, y_encoded['gender']))

def train_lightgbm_one_target(X, X_scaled, y_target, folds, target_name):
    is_binary = len(np.unique(y_target)) == 2
    oof_preds = np.zeros(len(X_scaled)) if is_binary else np.zeros((len(X_scaled), len(np.unique(y_target))))

    print(f"🎯 Start training {target_name}...")
    feature_importance_df = pd.DataFrame()

    for fold_id, (train_idx, val_idx) in enumerate(folds):
        print(f"📂 Fold {fold_id}:")
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_target[train_idx], y_target[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val)

        if target_name == 'gender':
            params = {'learning_rate': 0.17109161642272933,
                      'num_leaves': 239,
                      'max_depth': 16,
                      'min_child_samples': 32,
                      'feature_fraction': 0.9771,
                      'bagging_fraction': 0.5119,
                      'bagging_freq': 2,
                      'reg_alpha': 0.0071697,
                      'reg_lambda': 0.0012685}
        elif target_name == 'hold racket handed':
            params = {'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.05,
                      'boosting_type': 'gbdt', 'verbosity': -1, 'seed': 42}
        elif target_name == 'play years':
            params = {'objective': 'multiclass', 'metric': 'multi_logloss',
                      'learning_rate': 0.05, 'boosting_type': 'gbdt', 'verbosity': -1, 'seed': 42,
                      'class_weight': 'balanced', 'num_class': len(np.unique(y_target))}
        elif target_name == 'level':
            params = {'objective': 'multiclass', 'metric': 'multi_logloss',
                      'learning_rate': 0.05, 'boosting_type': 'gbdt', 'verbosity': -1, 'seed': 42,
                      'num_class': len(np.unique(y_target))}

        model = lgb.train(params, dtrain, valid_sets=[dtrain, dval],
                          num_boost_round=10000,
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)])

        preds = model.predict(X_val)
        oof_preds[val_idx] = preds if not is_binary else preds

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = X.columns
        fold_importance_df["importance"] = model.feature_importance()
        fold_importance_df["fold"] = fold_id
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    auc = roc_auc_score(y_target, oof_preds, average='micro') if is_binary else roc_auc_score(y_target, oof_preds, multi_class='ovr', average='micro')
    print(f"✅ {target_name}: Overall OOF AUC = {auc:.5f}")
    feature_importances[target_name] = feature_importance_df.groupby('feature')['importance'].mean().sort_values(ascending=False)
    return oof_preds, auc

In [6]:
# ====================
# 4. Feature Selection for Gender
# ====================
oof_gender, auc_gender = train_lightgbm_one_target(X, X_scaled, y_encoded['gender'], folds, 'gender')

# Select important features
selected_features = feature_importances['gender'][feature_importances['gender'].cumsum() <= feature_importances['gender'].sum() * 0.95].index.tolist()
X_selected = X[selected_features]
scaler = MinMaxScaler()
X_selected_scaled = scaler.fit_transform(X_selected)


🎯 Start training gender...
📂 Fold 0:
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003325 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38434
[LightGBM] [Info] Number of data points in the train set: 1564, number of used features: 159
[LightGBM] [Info] Start training from score 0.168159
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

In [8]:
# ====================
# 5. Add Play Years Special Features
# ====================
# === Play Years 特製新特徵 ===
def add_play_years_features_from_X(df):
    acc_magnitude = np.sqrt(df['Ax_mean_mean']**2 + df['Ay_mean_mean']**2 + df['Az_mean_mean']**2)
    gyro_magnitude = np.sqrt(df['Gx_mean_mean']**2 + df['Gy_mean_mean']**2 + df['Gz_mean_mean']**2)

    df['acc_energy_total'] = acc_magnitude ** 2
    df['gyro_energy_total'] = gyro_magnitude ** 2
    df['acc_vs_gyro_energy_ratio'] = acc_magnitude / (gyro_magnitude + 1e-6)
    df['acc_early_mean'] = df['acc_mag_mean_early_mean']
    df['acc_late_mean'] = df['acc_mag_mean_late_mean']
    df['early_late_energy_ratio'] = df['acc_early_mean'] / (df['acc_late_mean'] + 1e-6)
    return df

# ➔ 正確做法：從X裡important_features篩出來後再加特徵
X_selected_plus_play_years = add_play_years_features_from_X(X[important_features].copy())
X_selected_plus_play_years_scaled = scaler.fit_transform(X_selected_plus_play_years)


NameError: name 'important_features' is not defined

In [ ]:
# ====================
# 6. Train All Targets
# ====================
auc_scores = {}

# gender (important features)
oof_gender, auc_gender = train_lightgbm_one_target(X_selected, X_selected_scaled, y_encoded['gender'], folds, 'gender')
auc_scores['gender'] = auc_gender


In [ ]:
# hold racket handed (full features)
oof_hand, auc_hand = train_lightgbm_one_target(X, X_scaled, y_encoded['hold racket handed'], folds, 'hold racket handed')
auc_scores['hold racket handed'] = auc_hand

In [ ]:
# play years (play_year special features)
oof_years, auc_years = train_lightgbm_one_target(X_selected_plus_play_years, X_selected_plus_play_years_scaled, y_encoded['play years'], folds, 'play years')
auc_scores['play years'] = auc_years


In [ ]:
# level (full features)
oof_level, auc_level = train_lightgbm_one_target(X, X_scaled, y_encoded['level'], folds, 'level')
auc_scores['level'] = auc_level

In [ ]:
print("\nFinal AUCs:")
for k, v in auc_scores.items():
    print(f"{k}: {v:.5f}")

# Hyperparameter tuning with Optuna

In [ ]:
### 接續！幫你加上 自動找分數低的target自動Optuna調參 + Feature Engineering計畫

import optuna
import numpy as np
import lightgbm as lgb

# ==============================
# 🔥 這個函數是給Optuna調參用的
# ==============================
def optuna_objective(trial, X_train, X_val, y_train, y_val, is_binary, target_name):
    param = {
        'objective': 'binary' if is_binary else 'multiclass',
        'metric': 'auc' if is_binary else 'multi_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }
    if not is_binary:
        param['num_class'] = len(np.unique(y_train))

    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(param, dtrain, valid_sets=[dval], verbose_eval=False, early_stopping_rounds=50)

    preds = model.predict(X_val)
    if is_binary:
        score = roc_auc_score(y_val, preds)
    else:
        score = roc_auc_score(y_val, preds, multi_class='ovr', average='micro')

    return score


In [ ]:
# ==============================
# 🔥 這個是自動調參用的wrapper
# ==============================
def tune_target(X, y, folds, target_name, n_trials=300):
    is_binary = len(np.unique(y)) == 2

    for fold_id, (train_idx, val_idx) in enumerate(folds):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        break  # 用第一折來調參即可，避免過久

    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: optuna_objective(trial, X_train, X_val, y_train, y_val, is_binary, target_name), n_trials=n_trials)

    print("Best trial:", study.best_trial.params)
    return study.best_trial.params

In [ ]:
# ==============================
# 🔥 自動找AUC最低的target來改善
# ==============================
def auto_improve(X_all_scaled, y_encoded, folds, auc_scores, target_order=['gender', 'hold racket handed', 'play years', 'level']):
    sorted_targets = sorted(auc_scores.items(), key=lambda x: x[1])
    print("\n✅ Targets sorted by AUC:")
    for k, v in sorted_targets:
        print(f"{k}: AUC={v:.5f}")

    lowest_target = sorted_targets[0][0]
    print(f"\n🚀 Start tuning {lowest_target}...")

    tuned_params = tune_target(X_all_scaled, y_encoded[lowest_target].values, folds, lowest_target)
    print(f"\n✅ Tuning Finished for {lowest_target}! New best params:")
    print(tuned_params)

    return lowest_target, tuned_params

In [ ]:
# ✅ 直接接在剛剛完成版後面繼續

# === retrain with tuned params ===
retrained_auc_scores = {}

print("\n========================\n")
print("🎯 Retraining after tuning...")

for target in ['gender', 'hold racket handed', 'play years', 'level']:
    if target == worst_target:
        print(f"✅ {target} will use tuned params!")
        tuned_params = study.best_params

        # merge tuned params with basic settings
        final_params = {
            'objective': 'binary' if len(np.unique(y_encoded[target])) == 2 else 'multiclass',
            'metric': 'auc' if len(np.unique(y_encoded[target])) == 2 else 'multi_logloss',
            'boosting_type': 'gbdt',
            'verbosity': -1,
            'seed': 42,
        }
        final_params.update(tuned_params)

        if final_params['objective'] == 'multiclass':
            final_params['num_class'] = len(np.unique(y_encoded[target]))

        # retrain with tuned params
        oof_preds, auc = train_lightgbm_one_target_custom(X, X_scaled, y_encoded[target].values, folds, target, final_params)
        retrained_auc_scores[target] = auc

    else:
        # retrain normally
        oof_preds, auc = train_lightgbm_one_target(X, X_scaled, y_encoded[target].values, folds, target)
        retrained_auc_scores[target] = auc

In [ ]:

# === Compare before vs after tuning ===
print("\n🎯 Compare Old vs New AUC:")
for target in retrained_auc_scores:
    old_auc = auc_scores[target]
    new_auc = retrained_auc_scores[target]
    improve = new_auc - old_auc
    print(f"{target:20s} | Old AUC: {old_auc:.5f} | New AUC: {new_auc:.5f} | Change: {improve:+.5f}")

print("\n✅ Done!")


In [ ]:
# === Custom train function to allow passing custom params ===
def train_lightgbm_one_target_custom(X, X_scaled, y_target, folds, target_name, custom_params):
    is_binary = len(np.unique(y_target)) == 2

    if is_binary:
        oof_preds = np.zeros(len(X_scaled))
    else:
        num_classes = len(np.unique(y_target))
        oof_preds = np.zeros((len(X_scaled), num_classes))

    print(f"🎯 [Custom] Start training {target_name}...")

    feature_importance_df = pd.DataFrame()

    for fold_id, (train_idx, val_idx) in enumerate(folds):
        print(f"📂 Fold {fold_id}:")

        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y_target[train_idx], y_target[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dval = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(
            custom_params,
            dtrain,
            num_boost_round=10000,
            valid_sets=[dtrain, dval],
            valid_names=['train', 'valid'],
            callbacks=[
                lgb.early_stopping(50),
                lgb.log_evaluation(100)
            ]
        )

        if is_binary:
            preds = model.predict(X_val)
            oof_preds[val_idx] = preds
        else:
            preds = model.predict(X_val)
            oof_preds[val_idx] = preds

    if is_binary:
        auc = roc_auc_score(y_target, oof_preds, average='micro')
    else:
        auc = roc_auc_score(y_target, oof_preds, multi_class='ovr', average='micro')

    print(f"✅ [Custom] {target_name}: Overall OOF AUC = {auc:.5f}")

    return oof_preds, auc